In [38]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()  # reads .env
api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=api_key)

In [39]:
def score_tone(speech_text):
    prompt = f"""
    You are a financial policy analyst. Given the Federal Reserve speech below,
    rate the speaker’s tone toward each of the following topics on a continuous scale
    from -1 (dovish) to +1 (hawkish). Return only six numbers in JSON format
    with keys exactly as shown below.

    Topics and scoring guide:
    - Fed Funds Rate: hawkish = likely rate hike; dovish = likely rate cut
    - Labor Market: hawkish = strong labor market; dovish = weak labor market
    - Inflation: hawkish = high or concerning inflation; dovish = low or less concerning inflation
    - Real Activity: hawkish = strong real activity; dovish = weak real activity
    - Financial Stability: hawkish = stable system; dovish = less stable system
    - Balance Sheet: hawkish = QT or balance sheet reduction; dovish = QE or expansion

    Respond ONLY with JSON like this:
    {{
      "Fed Funds Rate": 0.4,
      "Labor Market": -0.2,
      "Inflation": 0.6,
      "Real Activity": 0.1,
      "Financial Stability": 0.3,
      "Balance Sheet": -0.5
    }}

    --- Speech ---
    {speech_text}
    """

    response = client.chat.completions.create(
        model="gpt-5",
        messages=[{"role": "user", "content": prompt}],
    )
    return response.choices[0].message.content


In [40]:
import json
def json_by_id(select_id):
    with open("text_data_old/logan.json", "r", encoding="utf-8") as f:
        text_data = json.load(f)
        selected_text = list(filter(lambda x: x["id"]==select_id, text_data))
        return selected_text[0]

def read_all_ids():
    with open("text_data_old/logan.json", "r", encoding="utf-8") as f:
        text_data    = json.load(f)
        selected_ids = list(map(lambda x: x["id"], text_data))
        return selected_ids

In [42]:
ids = read_all_ids()

data = []
for text_id in ids:
    json_result = json_by_id(text_id)
    tone_score = score_tone(json_result["text"])
    tone_score = json.loads(tone_score)
    result = {"id": text_id}
    result["gpt-5"] = tone_score
    print(result)
    data.append(result)

with open("info_folder/score.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)


{'id': 'feddsp:95906', 'gpt-5': {'Fed Funds Rate': 0.0, 'Labor Market': 0.0, 'Inflation': 0.0, 'Real Activity': 0.0, 'Financial Stability': -0.5, 'Balance Sheet': 0.6}}
{'id': 'feddsp:99102', 'gpt-5': {'Fed Funds Rate': -0.3, 'Labor Market': 0.15, 'Inflation': 0.3, 'Real Activity': 0.5, 'Financial Stability': 0.2, 'Balance Sheet': 0.4}}
{'id': 'fednsp:92982', 'gpt-5': {'Fed Funds Rate': -0.1, 'Labor Market': 0.0, 'Inflation': 0.0, 'Real Activity': -0.2, 'Financial Stability': 0.3, 'Balance Sheet': -0.6}}
{'id': 'feddsp:95519', 'gpt-5': {'Fed Funds Rate': 0.6, 'Labor Market': 0.8, 'Inflation': 0.9, 'Real Activity': 0.3, 'Financial Stability': 0.4, 'Balance Sheet': 0.8}}
{'id': 'feddsp:97129', 'gpt-5': {'Fed Funds Rate': 0.4, 'Labor Market': 0.75, 'Inflation': 0.65, 'Real Activity': 0.65, 'Financial Stability': 0.3, 'Balance Sheet': 0.8}}
{'id': 'feddsp:96051', 'gpt-5': {'Fed Funds Rate': 0.3, 'Labor Market': 0.6, 'Inflation': 0.7, 'Real Activity': 0.0, 'Financial Stability': -0.4, 'Bala